In [2]:
from mpl_toolkits.mplot3d import Axes3D
from rtree import index
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

import keras
from keras.models import load_model
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from keras.utils import CustomObjectScope
from keras.initializers import glorot_uniform

Using TensorFlow backend.


In [3]:
%store -r mc_data

In [158]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        xlayer_model = load_model('./models/xlayer_model.h5')

In [5]:
with CustomObjectScope({'GlorotUniform': glorot_uniform()}):
        track_classifier = load_model('./models/track_classifier.h5')

In [6]:
###### Data columns
i = 0
for val in mc_data.columns:
    print(i," ",val)
    i = i + 1

0   MCParticleID
1   MC_Hit_X
2   MC_Hit_Y
3   MC_Hit_Z
4   PrHit_LHCbID
5   PrHit_Xat0
6   PrHit_Zat0
7   PrHit_isX
8   PrHit_planeCode
9   PrHit_w2
10   PrHit_yMax
11   PrHit_yMin
12   PrHit_zone
13   event


In [8]:
mc_hits = mc_data.values

In [9]:
%store -r zLays
%store -r max_x
%store -r min_x
%store -r events

In [10]:
%store -r idx

In [11]:
print(events)

[15837 15838 15839 15840 15841 15842 15843 15844 15845 15846 15847 15848
 15849 15850 15851 15852 15853 15854 15855 15856 15857 15858 15859 15860
 15861 15862 15863 15864 15865 15866 15867 15868 15869 15870 15871 15872
 15873 15874 15875 15876 15877 15878 15879 15880 15881 15882 15883 15884
 15885 15886 15887 15888 15889 15890 15891 15892 15893 15894 15895 15896
 15897 15898 15899 15900 15901 15902 15903 15904 15905 15906 15907 15908
 15909 15910 15911 15912 15913 15914 15915 15916 15917 15918 15919 15920
 15921 15922 15923 15924 15925 15926 15927 15928 15929 15930 15931 15932
 15933 15934 15935 15936 15937 15938 15939 15940 15941 17909 17910 17911
 17912 17913 17914 17915 17916 17917 17918 17919 17920 17921 17922 17923
 17924 17925 17926 17927 17928 17929 17930 17931 17932 17933 17934 17935
 17936 17937 17938 17939 17940 17941 17942 17943 17944 17945 17946 17947
 17948 17949 17950 17951 17952 17953 17954 17955 17956 17957 17958 17959
 17960 17961 17962 17963 17964 17965 17966 17967 17

In [12]:
event_threshold = 15840

In [133]:
hits_total = mc_hits[mc_hits[:,13] < event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), float(hit[5][i]))

Total hits with threshold 631


In [128]:
hits_total = mc_hits[mc_hits[:,13] > event_threshold]

In [132]:
idx_total = idx 

In [140]:
from random import shuffle
X_classifier = []
Y_classifier = []

total_found = 0
total = 0
long_range = 300

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        layers = []
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
        
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            found = sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10]

            for i in range(0,len(found)):
                if (found[i] in X_final[1]):
                    found_bool += 1
                    break
                
            layers.append(found)
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            found = sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10]

            for i in range(0,len(found)):
                if (found[i] in X_final[2]):
                    found_bool += 1
                    break
                    
            layers.append(found)
            
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            found = sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10]

            for i in range(0,len(found)):
                if (found[i] in X_final[3]):
                    found_bool += 1
                    break

            layers.append(found)
            
        if (found_bool == 3):
            total_found += 1
            mean = ((X_final[0][0] + predicted[0] + predicted[1] + predicted[2]) / 4)
            for i in range(0,3):
                while(len(layers[i]) != 10):
                    layers[i].append(0)
                    
                shuffle(layers[i])
                pos = np.array(layers[i])
                print(pos)
                print(i + 1)
                print(np.where(pos == X_final[i + 1]))
                print(X_final[i + 1])
                layers[i].append(i + 1)
                layers[i].append(mean)
                
                X_classifier.append(layers[i])
                arr = np.zeros(10)
                arr[int(found_layers[i])] = 1
                Y_classifier.append(arr)
            break
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

[1549.54541016 1352.84692383 1472.88378906 1439.43151855 1156.14855957
 1220.92004395 1529.62475586 1440.43530273 1299.85046387 1180.31555176]
1


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [94]:
Y_classifier

[array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 1., 0.]),
 array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]),


In [113]:
track_selector = Sequential()
track_selector.add(Dense(12, activation='relu', input_dim=12))
track_selector.add(Dense(8, activation='relu'))
track_selector.add(Dense(10, activation='sigmoid'))
track_selector.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [114]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X_classifier), np.array(Y_classifier))
print("X Train length: %d" % len(X_train))
print("X Test Length %d" % len(X_test))
print("Y Train length: %d" % len(y_train))
print("Y Test length: %d" % len(y_test))

X Train length: 199300
X Test Length 66434
Y Train length: 199300
Y Test length: 66434


In [115]:
track_selector.fit(X_train, y_train, epochs=10, batch_size=100, verbose=1, validation_split=0.1)

Train on 179370 samples, validate on 19930 samples
Epoch 1/10
179370/179370 [==============================] - 6s 32us/sample - loss: 1.9610 - acc: 0.8722 - val_loss: 1.4604 - val_acc: 0.8962
Epoch 2/10
179370/179370 [==============================] - 5s 30us/sample - loss: 1.2630 - acc: 0.8958 - val_loss: 1.0153 - val_acc: 0.9002
Epoch 3/10
179370/179370 [==============================] - 5s 30us/sample - loss: 0.9283 - acc: 0.8958 - val_loss: 0.8743 - val_acc: 0.8998
Epoch 4/10
179370/179370 [==============================] - 5s 30us/sample - loss: 0.7766 - acc: 0.8939 - val_loss: 0.6595 - val_acc: 0.8901
Epoch 5/10
179370/179370 [==============================] - 5s 30us/sample - loss: 0.4994 - acc: 0.8920 - val_loss: 0.3683 - val_acc: 0.8982
Epoch 6/10
179370/179370 [==============================] - 5s 30us/sample - loss: 0.3447 - acc: 0.8971 - val_loss: 0.3194 - val_acc: 0.8999
Epoch 7/10
179370/179370 [==============================] - 5s 30us/sample - loss: 0.3133 - acc: 0.8989

In [116]:
score, acc = track_selector.evaluate(X_test, y_test, batch_size=25)
print('Test score:', score)
print('Test accuracy:', acc)

66434/66434 [==============================] - 4s 53us/sample - loss: 0.2855 - acc: 0.9010
Test score: 0.2855164923349617
Test accuracy: 0.900972


In [81]:
track_selector.save("./models/track_selector.h5")

In [83]:
hits_total = mc_hits[mc_hits[:,13] < event_threshold]
print("Total hits with threshold %d" % (len(hits_total)))
event = 0
idx = {}
actual = 0
for hit in hits_total:
    if (not hit[13] in idx):
        p = index.Property()
        p.dimension = 3
        idx[hit[13]] = index.Index(properties=p, interleaved=False)
    
    if (len(hit[5]) > 1):
        for i in range(0,len(hit[5])):
            idx[hit[13]].insert(int(hit[4][i]), (float(hit[5][i]), float(hit[5][i]), hit[8][i], hit[8][i], hit[12][i], hit[12][i]), float(hit[5][i]))

Total hits with threshold 631


In [164]:
import datetime as dt
X_test = []

total_found = 0
total = 0
long_range = 300

hits_total = mc_hits[mc_hits[:,13] < event_threshold]

for particle in hits_total:
    
    if (len(particle[5]) > 1):
        
        found_bool = 0
        found_layers = np.zeros(3)
        # Extract only 12 hits, as some of them are really close
        zone = particle[12][0]
        X_final = {}
        actual = -1
        layers = []
        
        for i in range(0, len(particle[5])):
            if (actual != particle[8][i]):
                X_final[particle[8][i]] = []
                X_final[particle[8][i]].append(particle[5][i])
                actual = particle[8][i]
            else:
                X_final[actual].append(particle[5][i])
                
        if (not 0 in X_final):
            continue
            
        predicted = (xlayer_model.predict(np.array([X_final[0][0], 0]).reshape(1,-1)))[0]
                
        if (1 in X_final):
            found = list(idx[particle[13]].intersection((predicted[0] - long_range, predicted[0] + long_range, 1, 1,zone,zone),objects='raw'))
            layers.append(sorted(found, key=lambda x: np.abs(x - predicted[0]))[:10])
            found_bool += 1
            
        if (2 in X_final):
            found = list(idx[particle[13]].intersection((predicted[1] - long_range, predicted[1] + long_range, 2, 2, zone,zone),objects='raw'))
            layers.append(sorted(found, key=lambda x: np.abs(x - predicted[1]))[:10])
            found_bool += 1
            
        if (3 in X_final):
            found = list(idx[particle[13]].intersection((predicted[2] - long_range, predicted[2] + long_range, 3, 3, zone, zone),objects='raw'))
            layers.append(sorted(found, key=lambda x: np.abs(x - predicted[2]))[:10])
            found_bool += 1
            
        if (found_bool == 3):
            retries = 0
            
            mean = ((X_final[0][0] + predicted[0] + predicted[1] + predicted[2]) / 4)
            
            for i in range(0,3):
                while(len(layers[i]) != 10):
                    layers[i].append(0)
                layers[i].append(i)
                layers[i].append(mean)
            
            select = ()
            total_found += 1
            if total_found == 4:
                print(predicted)
                print(X_final)
                print(layers[0])
                print(layers[1])
                print(layers[2])
                print(mean)
                print(sorted(layers[2], key=lambda x: np.abs(x - predicted[2])))
                print(layers[2][0])
                print(layers[2][5])
                print(predicted[3])
            #print(total_found/354)
            #print(total_val)
            #print('\n')
            
        total += 1

print("Mean found %f" % (total_found/total))
print("Total %d" % total)

[106.46674  114.496056 116.19107 ]
{0: [86.7600021362], 1: [174.5431976318], 2: [38.4754219055], 3: [127.4600067139], 4: [221.3099975586], 5: [315.7254333496, 316.3528442383], 6: [168.3948059082], 7: [264.0100097656], 8: [361.4849853516], 9: [463.0661315918], 10: [303.3332824707], 11: [405.4150085449]}
[157.2473907471, 170.2769775391, 174.5431976318, 232.1825408936, -19.8214054108, -20.4487953186, -72.8281097412, -112.6797637939, 369.9869384766, -182.5958099365, 0, 105.97846794127656]
[142.7221221924, 151.9773254395, 38.4754219055, 246.0904541016, 255.4711608887, 320.1171569824, 321.6228637695, -102.014175415, -139.6072387695, -140.2346191406, 1, 105.97846794127656]
[127.4600067139, 156.7850036621, 164.9100036621, 270.8649902344, -45.9650001526, 348.8599853516, -130.2100067139, -139.3150024414, 377.2149963379, 383.8099975586, 2, 105.97846794127656]
105.97846794127656
[105.97846794127656, 127.4600067139, 156.7850036621, 164.9100036621, 2, 270.8649902344, -45.9650001526, 348.8599853516, 

IndexError: index 3 is out of bounds for axis 0 with size 3

In [143]:
test_arr = [1330.2399902344, 1314.3850097656, 1313.7600097656, 1301.5400390625, 1440.8100585938, 1441.8100585938, 1442.8100585938, 1247.3599853516, 1487.6149902344, 1170.9899902344, 1, 1335.7846984863218]

track_selector.predict(np.array(test_arr).reshape(1,-1))

array([[0.18809891, 0.16985437, 0.15053976, 0.14337978, 0.12788728,
        0.05019951, 0.0556058 , 0.08065674, 0.01568136, 0.02650532]],
      dtype=float32)